# film_details
To get the general feel for a film, we'll define some baseline characteristics. Some of these, such as conversation speed, can be compared to individual scenes; this helps us find scenes which might be particularly dramatic or emotional.

In [1]:
import sys
sys.path.append('../data_serialization')
from serialization_preprocessing_io import *

In [2]:
film = 'plus_one_2019'
srt_df, subtitle_df, sentence_df, vision_df, face_df = read_pickle(film)

### Disregarding credits
First, we'll need a way to disregard frames that contain the credits. This will throw off our baseline characteristics.

As a quick-and-dirty way of accomplishing this, we can look for the last frame that has a face in it, and then disregard every frame after that. This is available as `generate_nocreds_dfs()`.

In [3]:
frame_before_credits = face_df[face_df['faces_found'] > 0].tail(1).index[0]  # final frame before credits
vision_nocreds_df = vision_df[0:frame_before_credits].copy()
face_nocreds_df = face_df[0:frame_before_credits].copy()

## Emotion
### Upset Percentage
We can try and get a feel for overall emotion of the film by looking at the emotion in dialogue scenes. We can count the number of "sad" and "angry" faces are found, as a percentage of overall facial emotions. This is available as `get_upset_percentage()`.

In [4]:
try:
    sad_percentage = face_nocreds_df.p_emotion.value_counts(normalize=True)['sad']
except KeyError:
    sad_percentage = 0
try:
    angry_percentage = face_nocreds_df.p_emotion.value_counts(normalize=True)['angry']
except KeyError:
    angry_percentage = 0

upset_emotion_percentage = sad_percentage + angry_percentage
upset_emotion_percentage

0.45010845986984815

## Dialogue Vibe
### Profanity Per Word
First, we'll take a look at the dialogue. We'll first quantify how much profanity is in the film as a whole. We'll look for "profanity per word", like "1 in X words is a profanity". After we calculate this, we'll be able to search for dramatic scenes with an above-average amount of profanity.

First, we'll count all the words in spoken sentences. Then we'll count up the number of profanities.

In [5]:
space_count = 0
sentence_list = sentence_df.sentence.tolist()
for sentence in sentence_list:
    for character in sentence:
        if character.isspace():
            space_count += 1
word_count = space_count + len(sentence_list)

In [6]:
profanity_count = sentence_df.profanity.sum()
profanity_per_word = profanity_count/word_count
profanity_per_word

0.008517324071777061

In [7]:
if profanity_per_word == 0:
    print('The film contains no profanity.')
else:
    print('One in', round(1 / profanity_per_word), 'words is a profanity.')

One in 117 words is a profanity.


This is available as a function `get_profanity_per_word()`.

### Cadence
The speed of conversation is an important metric. Witty comedies might have characters launch lightning-fast insults at each other. Introspective dramas or quiet romances might have very little dialogue. We can measure cadence by counting the sentences per minute.

In [8]:
round(len(sentence_df) / (len(vision_df) / 60))

31

### Words per Sentence
Dramas may have lengthy, wordy monologues. Comedies may contain rapid-fire arguments. We can take a look at words per sentence.

In [9]:
space_count = 0
sentence_list = sentence_df.sentence.tolist()
for sentence in sentence_list:
    for character in sentence:
        if character.isspace():
            space_count += 1

word_count = space_count + len(sentence_list)

words_per_sentence = word_count / len(sentence_list)
words_per_sentence

4.0043046357615895

# Vision Style
There are a few technical computer vision details that may be important.

In [10]:
print('Aspect Ratio:', vision_nocreds_df.aspect_ratio.value_counts().index[0])
print('Average shot duration:', round(len(vision_nocreds_df) / vision_nocreds_df.shot_id.max(), 2))
print('Average frame brightness:', round(vision_nocreds_df.brightness.mean()))
print('Average frame contrast:', round(vision_nocreds_df.contrast.mean()))

Aspect Ratio: 2.39
Average shot duration: 3.73
Average frame brightness: 59
Average frame contrast: 44


Many of these statistics are bundled together in the function `display_film_baseline()`.